<a href="https://colab.research.google.com/github/Gjeffroy/LLM_finetuning/blob/main/Evaluating_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install openai asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 1.4 MB/s eta 0:00:00


In [1]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_API_key')

In [2]:
import pandas as pd
# load df
df = pd.read_csv('df_reponse_orientees_mistral.csv', sep = ';')

In [18]:
df[['article_id', 'question_orientée', 'reponse_ministral_8b', 'reponse_ministral_3b', 'texte']]

,article_id,question_orientée,reponse_ministral_8b,reponse_ministral_3b,texte
0,L101-1,Quels acteurs sont désignés comme gestionnaire...,Les collectivités publiques sont désignées com...,Les collectivités publiques sont les gestionna...,Le territoire français est le patrimoine commu...
1,L101-2,Quels sont les objectifs du développement dura...,Les objectifs du développement durable visés p...,Les objectifs du développement durable visés p...,Dans le respect des objectifs du développement...
2,L101-2-1,Quels sont les critères à prendre en compte po...,Les critères à prendre en compte pour atteindr...,Les critères à prendre en compte pour atteindr...,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Quels sont les territoires exclus de la réglem...,Les collectivités d'outre-mer régies par l'art...,Les territoires exclus de la réglementation de...,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Quelles sont les deux conditions requises pour...,Les deux conditions requises pour qu'un projet...,Le projet doit être destiné à l'une des activi...,L'autorité administrative compétente de l'Etat...
...,...,...,...,...,...
2412,A614-2,Quel est le montant de la rémunération au titr...,La rémunération au titre de leur vacation jour...,La rémunération au titre de leur vacation jour...,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Quels sont les crédits sur lesquels sont imput...,Les crédits des ministères concernés.,Les dépenses de rémunération et de transport d...,Les dépenses correspondants à la rémunération ...
2414,A614-4,Quelles sont les autorités compétentes pour mo...,Les dispositions des articles A. 614-1 à A. 61...,Les autorités compétentes pour modifier les di...,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Qu'est-ce que le contrôleur technique atteste ...,Le contrôleur technique atteste avoir fait con...,Le contrôleur technique atteste avoir fait con...,Attestation du contrôleur technique établissan...


In [4]:
df.columns

Index(['ref', 'title_main', 'texte', 'dateDebut', 'dateFin', 'num', 'id',
       'cid', 'type', 'etat', 'nota', 'version_article', 'ordre',
       'conditionDiffere', 'infosComplementaires', 'surtitre', 'nature',
       'texteHtml', 'dateFinExtension', 'versionPrecedente', 'refInjection',
       'idTexte', 'idTechInjection', 'origine', 'dateDebutExtension',
       'idEliAlias', 'cidTexte', 'sectionParentId', 'multipleVersions',
       'comporteLiensSP', 'sectionParentTitre', 'infosRestructurationBranche',
       'idEli', 'sectionParentCid', 'numeroBo',
       'infosRestructurationBrancheHtml', 'historique',
       'infosComplementairesHtml', 'renvoi', 'fullSectionsTitre', 'notaHtml',
       'inap', 'article_id', 'instruction', 'question_orientée',
       'reponse_ministral_3b', 'reponse_ministral_8b'],
      dtype='object')

In [5]:
from openai import AsyncOpenAI
import openai

client_async = AsyncOpenAI()

async def generate_response_async(prompt):

    messages = [{"role": "user", "content": prompt}]

    response = await client_async.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        timeout=1000
    )
    return response.choices[0].message.content

In [15]:

import asyncio
from tqdm.notebook import tqdm

texte_col = 'texte'
question_col = 'question_orientée'
reponse_col = 'reponse_ministral_3b'

async def main(df):

    # Define the instruction template
    instruction = """Tu es un assistant IA chargé d'évaluer les réponses fournies par des étudiants.
      La réponse doit être ancrée dans le TEXTE et pertinente par rapport à la QUESTION posée.
      Dans certains cas, le TEXTE peut faire référence à un autre article du code d'urbanisme. La question pourrait donc dépendre de ce contenu externe, ce qui peut influencer la pertinence de la réponse.

      # TEXTE :
      {texte}

      # QUESTION :
      {question}

      # REPONSE :
      {reponse}

      Renvoie un JSON contenant les éléments suivants :
        - "question": la question posée.
        - "informations_completes": une note binaire (true ou false) indiquant si toutes les informations nécessaires à la réponse sont présentes dans le TEXTE.
        - "note_confiance": une note de confiance pour la pertinence de la réponse, sur une échelle de 1 à 5.
        - "explication": une explication justifiant la note de confiance, avec les raisons spécifiques si la note est inférieure à 5.
    """


    # Prepare prompts for async API calls
    prompts = [instruction.format(texte=row[texte_col], question=row[question_col], reponse=row[reponse_col]) for _, row in df.iterrows()]

    # Create tasks for each prompt
    tasks = [generate_response_async(p) for p in prompts]

    # Initialize progress bar and run tasks with gather to maintain order
    results = []
    with tqdm(total=len(tasks)) as pbar:
        for result in await asyncio.gather(*tasks):
            results.append(result)
            pbar.update(1)  # Update progress after each task completes

    return results

In [16]:
# prompt: # run main by batch of 500 and wait 120s to avoid rate limite (dont repeat the code already wrfitten, just call main and filter df according to instruction)
import time

results = []
batch_size = 5
wait_time = 5
intervals_df = [(i, i + batch_size) if (i + batch_size) < len(df) else (i, len(df)) for i in range(0, len(df[0:5]), batch_size)]
for interval in intervals_df:
  df_batch = df.iloc[interval[0]:interval[1]]
  results += await main(df_batch)
  time.sleep(wait_time)

  0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
print('\n'.join(results))

{
    "question": "Quels acteurs sont désignés comme gestionnaires et garants du territoire français selon le texte ?",
    "informations_completes": true,
    "note_confiance": 5,
    "explication": "La réponse fournie est correcte et ancrée dans le texte. En effet, le texte indique clairement que les collectivités publiques sont désignées comme les gestionnaires et les garantes du territoire français. Il n'y a pas de raison de douter de la pertinence de la réponse."
}
{
    "question": "Quels sont les objectifs du développement durable visés par l'action des collectivités publiques en matière d'urbanisme ?",
    "informations_completes": true,
    "note_confiance": 5,
    "explication": "Toutes les informations requises pour répondre à la question sont clairement présentées dans le TEXTE. Les objectifs du développement durable en matière d'urbanisme sont explicitement énumérés, et la réponse fournie reflète fidèlement ces objectifs sans omission ni interprétation erronée."
}
{
    "q

In [37]:
df['question_orientée'][19]

"Quelles sont les dispositions à appliquer aux décisions des autorités publiques ayant une incidence sur l'environnement ?"